In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
from dlomix.models import PrositIntensityPredictor
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
import tensorflow as tf
from tensorflow import keras
import wandb
from wandb.keras import WandbCallback
import keras_nlp
print([x for x in dir(dlomix) if not x.startswith("_")])

2023-06-16 19:57:49.748420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 19:57:49.855884: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-16 19:57:50.507144: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-16 19:57:50.507206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

['META_DATA', 'constants', 'data', 'eval', 'layers', 'losses', 'models', 'pipelines', 'reports', 'utils']


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
project_name = "transforming-prosit"

In [6]:
from dlomix.data import IntensityDataset

TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/Intensity/proteomeTools_train_val.csv'
BATCH_SIZE = 64
int_data = IntensityDataset(data_source=TRAIN_DATAPATH, seq_length=30,
                            collision_energy_col='collision_energy', batch_size=BATCH_SIZE, val_ratio=0.2, test=False)

2023-06-16 19:57:59.672807: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-16 19:58:00.071106: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43210 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [7]:
wandb.init(project=project_name, name='baseline')

model = PrositIntensityPredictor(seq_length=30,embedding_output_dim=16,
        recurrent_layers_sizes=(256, 512))

wandb: Currently logged in as: mamisashvili-lizi. Use `wandb login --relogin` to force relogin


In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, 
              loss=masked_spectral_distance, metrics=[masked_pearson_correlation_distance])

In [9]:
%%time
history = model.fit(int_data.train_data, validation_data=int_data.val_data,
                    epochs=60, callbacks=[WandbCallback()])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/60


2023-06-16 19:59:55.377593: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-16 19:59:56.499514: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8801


366/369 [============================>.] - ETA: 0s - loss: 0.6035 - masked_pearson_correlation_distance: 0.4752WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor. Received: inputs=[<tf.Tensor 'prosit_intensity_predictor/Cast:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 6) dtype=float32>]. Consider rewriting this model with the Functional API.


wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


...

369/369 [==============================] - 17s 45ms/step - loss: 0.2270 - masked_pearson_correlation_distance: 0.2004 - val_loss: 0.2373 - val_masked_pearson_correlation_distance: 0.2047
Epoch 58/60
369/369 [==============================] - 4s 12ms/step - loss: 0.2280 - masked_pearson_correlation_distance: 0.2026 - val_loss: 0.2435 - val_masked_pearson_correlation_distance: 0.2080
Epoch 59/60
369/369 [==============================] - 4s 12ms/step - loss: 0.2288 - masked_pearson_correlation_distance: 0.2028 - val_loss: 0.2382 - val_masked_pearson_correlation_distance: 0.2054
Epoch 60/60
364/369 [============================>.] - ETA: 0s - loss: 0.2257 - masked_pearson_correlation_distance: 0.1979

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230616_195944-t6k5lebf/files/model-best/assets


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230616_195944-t6k5lebf/files/model-best/assets
wandb: Adding directory to artifact (/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230616_195944-t6k5lebf/files/model-best)... Done. 0.2s


369/369 [==============================] - 16s 44ms/step - loss: 0.2255 - masked_pearson_correlation_distance: 0.1977 - val_loss: 0.2369 - val_masked_pearson_correlation_distance: 0.1996
CPU times: user 13min 47s, sys: 27.2 s, total: 14min 15s
Wall time: 14min 10s


In [11]:
model.summary()

Model: "prosit_intensity_predictor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 string_lookup (StringLookup  multiple                 0         
 )                                                               
                                                                 
 embedding (Embedding)       multiple                  352       
                                                                 
 sequential (Sequential)     multiple                  4096      
                                                                 
 sequence_encoder (SequenceE  multiple                 1630544   
 ncoder)                                                         
                                                                 
 encoder_att (AttentionLayer  multiple                 542       
 )                                                               
                                        